# Installing Libraries and Imports

In [9]:
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In this notebook, you will be working with a Large Language Model (LLM) and explore its capabilities to help you solve various problems.

# Loading Model

We will be using Phi-3 as our LLM.

In [2]:
MODEL_ARGS = {
    'Name': 'microsoft/Phi-3-mini-128k-instruct',
    'DType': torch.bfloat16 # add torch.
}
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def load_model(model_args):


    model = AutoModelForCausalLM.from_pretrained(
        model_args['Name'],
        trust_remote_code=True,
        torch_dtype=model_args['DType'], #remove torch.
        low_cpu_mem_usage=True,
        device_map={"": device},
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args['Name'],
        trust_remote_code=True,
    )

    return model, tokenizer

In [4]:
model, tokenizer = load_model(MODEL_ARGS)

config.json:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

c:\Users\Mahdi\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mahdi\.cache\huggingface\hub\models--microsoft--Phi-3-mini-128k-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

KeyboardInterrupt: 

# First Inference

In [49]:
def generate_text(model, tokenizer, prompt, max_new_tokens = 100, do_sample=True, temperature=0.5):

    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    if do_sample:
        output_ids = model.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature)
    else:
        output_ids = model.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=do_sample)

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output_text[len(prompt):]

Lets break down this function:

**Arguments**:

* **model**: The language model used for text generation.
* **tokenizer**: The tokenizer that converts text to tokens and vice versa.
* **prompt**: The initial text input that the model will build upon.
* **max_new_tokens**: The maximum number of new tokens to generate.
* **do_sample**: Whether to sample the next token or use deterministic decoding.
* **temperature**: Controls the randomness of sampling; higher values produce more diverse outputs. ( model creativity )

**Functionality**:

The generate_text function creates more text based on a given starting prompt using a language model and tokenizer. It first converts the prompt into tokens (numbers the model understands), then generates additional tokens to continue the text. Depending on settings, it can generate text randomly or in a fixed way. Finally, it converts the tokens back into readable text and returns the part that extends beyond the original prompt.

## Without template

In [29]:
prompt = """Tell me a funny story about a cute cat"""

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=200,
    temperature=1.2,
)

''

## With template

In [31]:
prompt = """Insturction: Tell me a funny story about a cute cat
Answer:"""

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=200,
    temperature=1.2,
)

" Once upon a time, there was a cat named Misty who loved to play pranks around her family. One day, Misty decided she wanted to surprise everyone with a hilarious cat-napped hat made from stolen yarn! Her owners were none the wiser as they knitted and shopped, until one evening when they saw their own heads wrapped up in soft, fluffy yarn, courtesy of their mischievous kitty. The family couldn't stop laughing, even Misty herself seemed to purr with delight at the uproarious laughter. They all declared she was the 'Queen of Comedy' from that day forward, and who could blame her? Any cat could charm their way to a room full of hilarious memories, right in Misty's adventurous paws."

As you can see, the output generated by these models depends on the prompt provided.

But that's just the beginning! Let's try different prompt layouts

( You can use the keyword "Prompt Engineering" for more information )

# In Context Learning ( ICL )

LLMs can learn from their prompts, as you can give it examples or guide it and teach it how to solve the problem.

## Learning from examples

### No example

In [37]:

prompt = """Question: John volunteers at a shelter once a week for 3 hours at a time. How many hours does he volunteer per year?
Answer:"""

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=200,
    temperature=0.2,
)

' John volunteers 192 hours per year.'

The right answer is ( 12 * 4 ) * 3 = 144

### One Example

In [51]:

prompt = """Question: John volunteers at a shelter once a week for 7 hours at a time. How many hours does he volunteer per year?
Answer: John volunteers 336 hours per year.
Question: John volunteers at a shelter once a week for 3 hours at a time. How many hours does he volunteer per year?
Answer:"""

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=10,
    do_sample=False,
    temperature=0.0,
)

' John volunteers 156 hours per year'

Examples are not always effective for mathematical problems, so let's try another method.

## Chain of Thoughts ( CoT )

In [54]:

prompt = """Question: John volunteers at a shelter once a week for 7 hours at a time. How many hours does he volunteer per year?
Answer: There are 12 months in one year and 4 weeks in each month. So in one year, there are 12 * 4 = 48 weeks. If Jhon volunteers at a shelter once a week for 7 hours,
John volunteers 48 * 7 = 336 hours per year.
Question: John volunteers at a shelter once a week for 3 hours at a time. How many hours does he volunteer per year?
Answer:"""

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=70,
    do_sample=False,
    temperature=0.0,
)

' There are 12 months in one year and 4 weeks in each month. So in one year, there are 12 * 4 = 48 weeks. If John volunteers at a shelter once a week for 3 hours,\nJohn volunteers 48 * 3 = 144 hours per year.'

In Chain of Thought (CoT), we guide the model with one or more examples and provide it with the steps to solve the problem.